# VGG
VGG16相比AlexNet的一个改进是采用连续的几个3x3的卷积核代替AlexNet中的较大卷积核（11x11，7x7，5x5）。对于给定的感受野（与输出有关的输入图片的局部大小），采用堆积的小卷积核是优于采用大的卷积核，因为多层非线性层可以增加网络深度来保证学习更复杂的模式，而且代价还比较小（参数更少）。

## 导入库

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## 超参
如果要使vgg更深只需修改CONV_ARCH

In [2]:
EPOCHS = 50
BATCH_SIZE = 16
IMAGE_SIZE = 224
LR = 0.05
VERBOSE = 1
CONV_ARCH = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

## 导入数据集

In [3]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

In [4]:
train_images = tf.reshape(train_images, (train_images.shape[0], train_images.shape[1], train_images.shape[2], -1))
test_images = tf.reshape(test_images, (test_images.shape[0], test_images.shape[1], test_images.shape[2], -1))

In [5]:
train_images = tf.image.resize_with_pad(train_images, IMAGE_SIZE, IMAGE_SIZE)
test_images = tf.image.resize_with_pad(test_images, IMAGE_SIZE, IMAGE_SIZE)

In [6]:
train_images = tf.cast(train_images, dtype=tf.float32) / 255.
test_images = tf.cast(test_images, dtype=tf.float32) / 255.

In [7]:
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.int32)

## 定义模型

In [8]:
def vgg_block(num_convs, num_channels):
    blk = keras.models.Sequential()
    for _ in range(num_convs):
        blk.add(keras.layers.Conv2D(num_channels,kernel_size=3,padding='same',activation='relu'))
    blk.add(keras.layers.MaxPool2D(pool_size=2, strides=2))
    return blk

In [9]:
def vgg(conv_arch):
    model = keras.models.Sequential()
    model.add(keras.Input((224, 224, 1)))
    for (num_convs, num_channels) in conv_arch:
        model.add(vgg_block(num_convs, num_channels))
    model.add(keras.models.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(4096, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(4096, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='sigmoid')
    ]))
    return model

model = vgg(CONV_ARCH)
model.summary()

## 设置优化器和损失函数

In [10]:
optimizer = keras.optimizers.SGD(learning_rate=LR)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## 开始训练

In [ ]:
history = model.fit(train_images, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=VERBOSE)

Train on 10000 samples
Epoch 1/50
10000/10000 [==============================] - 105s 10ms/sample - loss: 2.3075 - accuracy: 0.1004
Epoch 2/50
10000/10000 [==============================] - 101s 10ms/sample - loss: 2.3051 - accuracy: 0.0977
Epoch 3/50
 3104/10000 [========>.....................] - ETA: 1:09 - loss: 2.3053 - accuracy: 0.0912

## 绘制梯度下降图

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 0.1)
    plt.show()
    
plot_learning_curves(history)

## 测试训练好的模型

In [ ]:
model.evaluate(test_images, test_labels)

## 保存模型 

In [ ]:
model.save('./save/save_model/model.h5')

In [ ]:
model.save_weights('./save/save_weights/model_weights.h5')